# Week 10: Marketing Mix Modeling (MMM)

**Goal:** Master Marketing Mix Modeling to measure aggregate channel impact and optimize budget allocation.

**Time Commitment:** ~1 hour per day × 7 days = 7 hours total

**What You'll Learn:**
- MMM fundamentals and use cases
- Data preparation for time-series regression
- Linear regression for marketing effectiveness
- Adstock transformations (lagged effects)
- Saturation curves (diminishing returns)
- Model validation and diagnostics
- Budget optimization using MMM

**Why This Matters:**
As a Marketing Measurement Partner, MMM helps you:
- Measure marketing effectiveness without user-level tracking
- Quantify incrementality at the aggregate level
- Understand diminishing returns and optimal spend levels
- Account for delayed effects of marketing (adstock)
- Optimize budgets across channels mathematically
- Work in privacy-first environments (no cookies needed)

MMM is experiencing a renaissance as privacy regulations limit user tracking. It's an essential skill for modern marketers.

---

## 📅 Day 64: MMM Fundamentals (~60 min)

### Learning Objectives
- Understand what Marketing Mix Modeling is
- Learn when MMM is appropriate vs other methods
- Understand the basic MMM equation
- Explore the components of an MMM

### The Business Problem
Your company spends millions across TV, digital, radio, and print. You need to measure the ROI of each channel and optimize the budget allocation—but you can't track individual users.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Settings
np.random.seed(42)
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

### 📖 Concept: What is Marketing Mix Modeling?

**MMM** is a statistical technique that uses regression analysis to measure the impact of marketing activities on sales/conversions.

**Basic Equation:**
```
Sales = β₀ + β₁×TV + β₂×Digital + β₃×Radio + β₄×Print + β₅×Seasonality + ε
```

Where:
- **Sales**: Target variable (conversions, revenue, units sold)
- **β coefficients**: Impact of each marketing channel
- **Seasonality**: Control variables (holidays, trends, etc.)
- **ε**: Random error

**Key Features:**
- Uses **aggregate data** (weekly/daily totals, not user-level)
- Measures **incremental impact** of marketing
- Accounts for **delayed effects** (adstock)
- Models **diminishing returns** (saturation)
- Requires **historical data** (typically 2+ years of weekly data)

### 📖 Concept: MMM vs Other Methods

| Method | Level | User Tracking | Strengths | Limitations |
|--------|-------|---------------|-----------|-------------|
| **MMM** | Aggregate | No | Privacy-friendly, measures all channels, budget optimization | Requires lots of data, less granular |
| **Attribution** | User | Yes | Granular, journey-level | Privacy concerns, doesn't prove causality |
| **A/B Testing** | User | Yes | Proves causality | Only tests incremental changes |
| **Geo Experiments** | Geographic | No | Proves causality, privacy-friendly | Complex to run, limited scale |

In [ ]:
# Generate synthetic MMM data
def generate_mmm_data(n_weeks=104):
    """
    Generate 2 years of weekly marketing and sales data.
    """
    np.random.seed(42)
    
    # Date range
    dates = pd.date_range('2022-01-03', periods=n_weeks, freq='W-MON')
    
    # Seasonality (higher in Q4)
    weeks = np.arange(n_weeks)
    seasonality = 100 + 30 * np.sin(2 * np.pi * weeks / 52) + 20 * (weeks % 52 > 40)
    
    # Marketing spend (with some variation)
    tv_spend = 50000 + 20000 * np.random.randn(n_weeks) + 10000 * (weeks % 52 > 40)
    tv_spend = np.maximum(tv_spend, 10000)
    
    digital_spend = 30000 + 10000 * np.random.randn(n_weeks)
    digital_spend = np.maximum(digital_spend, 5000)
    
    radio_spend = 15000 + 5000 * np.random.randn(n_weeks)
    radio_spend = np.maximum(radio_spend, 2000)
    
    print_spend = 10000 + 3000 * np.random.randn(n_weeks)
    print_spend = np.maximum(print_spend, 1000)
    
    # True effects with diminishing returns (square root for saturation)
    tv_effect = 2.5 * np.sqrt(tv_spend)
    digital_effect = 3.0 * np.sqrt(digital_spend)
    radio_effect = 1.8 * np.sqrt(radio_spend)
    print_effect = 1.2 * np.sqrt(print_spend)
    
    # Sales = base + marketing effects + seasonality + noise
    base_sales = 5000
    sales = (base_sales + 
             tv_effect + digital_effect + radio_effect + print_effect +
             seasonality + 
             500 * np.random.randn(n_weeks))
    
    df = pd.DataFrame({
        'date': dates,
        'sales': sales,
        'tv_spend': tv_spend,
        'digital_spend': digital_spend,
        'radio_spend': radio_spend,
        'print_spend': print_spend
    })
    
    return df

# Generate data
df_mmm = generate_mmm_data()

print("Marketing Mix Modeling Dataset")
print("=" * 60)
print(f"Time period: {df_mmm['date'].min()} to {df_mmm['date'].max()}")
print(f"Weeks of data: {len(df_mmm)}")
print("\nFirst 10 rows:")
print(df_mmm.head(10))

print("\nSummary Statistics:")
print(df_mmm.describe())

### 💡 Try It: Exploratory Data Analysis

Visualize the relationships between marketing spend and sales.

In [ ]:
# YOUR CODE HERE
# Create visualizations:
# 1. Time series of sales over time
# 2. Time series of total marketing spend over time
# 3. Scatter plots: each marketing channel vs sales
# 4. Correlation matrix between all variables
# 5. What initial patterns do you notice?



### 📖 Concept: Simple Linear Model

Start with a basic linear regression to establish a baseline.

In [ ]:
# Prepare features and target
X = df_mmm[['tv_spend', 'digital_spend', 'radio_spend', 'print_spend']]
y = df_mmm['sales']

# Fit linear regression
model_simple = LinearRegression()
model_simple.fit(X, y)

# Predictions
y_pred = model_simple.predict(X)

# Metrics
r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))

print("Simple Linear Model Results")
print("=" * 60)
print(f"R² Score: {r2:.4f}")
print(f"MAE: ${mae:,.2f}")
print(f"RMSE: ${rmse:,.2f}")
print("\nCoefficients (ROI per $1 spent):")
for channel, coef in zip(X.columns, model_simple.coef_):
    print(f"  {channel:20s}: ${coef:.4f}")
print(f"\nIntercept (base sales): ${model_simple.intercept_:,.2f}")

### ✏️ Exercise 1: Interpret Model Coefficients

Translate the model coefficients into business insights.

In [ ]:
# YOUR CODE HERE
# Based on the model coefficients:
# 1. Which channel has the highest ROI per dollar spent?
# 2. Calculate total sales attributed to each channel (coefficient × total spend)
# 3. What percentage of total sales comes from marketing vs baseline?
# 4. If you had an extra $10,000 to spend, which channel should get it?
# 5. What are the limitations of this simple linear model?



### 🎯 Day 64 Mini-Project: Model Diagnostics

Check if the linear model's assumptions are met.

In [ ]:
# YOUR CODE HERE
# Perform model diagnostics:
# 
# 1. Residuals plot: actual vs predicted
# 2. Residuals over time (check for autocorrelation)
# 3. Q-Q plot (check normality of residuals)
# 4. Residuals vs fitted values (check homoscedasticity)
# 5. Check for multicollinearity (VIF - variance inflation factor)
# 
# What issues do you identify?
# Hint: You may need to import additional libraries like statsmodels



### 🎓 Day 64 Key Takeaways

✅ MMM measures aggregate marketing impact using regression  
✅ Works with weekly/daily data, no user tracking needed  
✅ Linear regression provides a baseline model  
✅ Model coefficients represent marginal ROI  
✅ Simple models miss adstock and saturation effects  

**Next:** Tomorrow we'll prepare data properly for MMM!

---

## 📅 Day 65: Data Preparation for MMM (~60 min)

### Learning Objectives
- Prepare time-series data for MMM
- Create control variables (seasonality, trends, events)
- Handle missing data and outliers
- Feature engineering for marketing data

### The Business Problem
Raw marketing data needs significant preparation. You must account for seasonality, holidays, promotions, and external factors to isolate true marketing effects.

### 📖 Concept: Control Variables

To measure marketing impact accurately, control for:
1. **Seasonality**: Weekly, monthly, quarterly patterns
2. **Trends**: Long-term growth or decline
3. **Events**: Holidays, promotions, competitor actions
4. **Exogenous factors**: Economy, weather, COVID, etc.

Without controls, you might attribute seasonal sales spikes to marketing.

In [ ]:
# Add date features
df_mmm['year'] = df_mmm['date'].dt.year
df_mmm['month'] = df_mmm['date'].dt.month
df_mmm['quarter'] = df_mmm['date'].dt.quarter
df_mmm['week_of_year'] = df_mmm['date'].dt.isocalendar().week

# Create trend variable (weeks since start)
df_mmm['trend'] = np.arange(len(df_mmm))

# Create holiday indicators
# Black Friday/Cyber Monday (week 47-48 typically)
df_mmm['is_black_friday'] = ((df_mmm['week_of_year'] >= 47) & 
                              (df_mmm['week_of_year'] <= 48)).astype(int)

# Christmas season (weeks 50-52)
df_mmm['is_christmas'] = (df_mmm['week_of_year'] >= 50).astype(int)

# Summer (weeks 22-35)
df_mmm['is_summer'] = ((df_mmm['week_of_year'] >= 22) & 
                        (df_mmm['week_of_year'] <= 35)).astype(int)

# Create cyclical features for seasonality
df_mmm['week_sin'] = np.sin(2 * np.pi * df_mmm['week_of_year'] / 52)
df_mmm['week_cos'] = np.cos(2 * np.pi * df_mmm['week_of_year'] / 52)

print("Enhanced Dataset with Control Variables:")
print(df_mmm[['date', 'sales', 'trend', 'is_black_friday', 'week_sin', 'week_cos']].head(10))

### 💡 Try It: Visualize Seasonality

Explore seasonal patterns in the data.

In [ ]:
# YOUR CODE HERE
# 1. Calculate average sales by month
# 2. Create a seasonal decomposition plot (trend, seasonal, residual)
# 3. Compare sales during Black Friday weeks vs non-holiday weeks
# 4. Plot sales over time with holiday periods highlighted
# 5. How much of sales variation is due to seasonality vs marketing?
#
# Hint: You can use statsmodels.tsa.seasonal_decompose



### 📖 Concept: Standardization

Standardize features so coefficients are comparable across channels with different spend levels.

In [ ]:
# Prepare feature sets
marketing_channels = ['tv_spend', 'digital_spend', 'radio_spend', 'print_spend']
control_vars = ['trend', 'week_sin', 'week_cos', 'is_black_friday', 'is_christmas', 'is_summer']

# Standardize marketing spend
scaler = StandardScaler()
df_mmm[marketing_channels + control_vars] = scaler.fit_transform(df_mmm[marketing_channels + control_vars])

print("Standardized Features (mean=0, std=1):")
print(df_mmm[marketing_channels + control_vars].describe())

### ✏️ Exercise 2: Model with Controls

Build a model that includes control variables and compare to baseline.

In [ ]:
# YOUR CODE HERE
# 1. Build Model 1: Only marketing channels (baseline)
# 2. Build Model 2: Marketing channels + control variables
# 3. Compare R² scores
# 4. How do marketing coefficients change when controls are added?
# 5. Which model gives more accurate marketing attribution?
# 6. Why is Model 2 better?



### 🎯 Day 65 Mini-Project: Feature Engineering Pipeline

Create a reusable data preparation pipeline for MMM.

In [ ]:
# YOUR CODE HERE
# Build a function that takes raw MMM data and:
# 1. Adds date features (year, month, quarter, week)
# 2. Creates trend variable
# 3. Adds holiday indicators (you choose which ones)
# 4. Creates cyclical features (sin/cos)
# 5. Handles missing values
# 6. Removes outliers (optional)
# 7. Standardizes features
# 8. Returns clean DataFrame ready for modeling
#
# Test it on a new dataset



### 🎓 Day 65 Key Takeaways

✅ Control variables isolate true marketing effects  
✅ Seasonality and trends must be accounted for  
✅ Holiday indicators capture special events  
✅ Standardization makes coefficients comparable  
✅ Proper data prep dramatically improves model accuracy  

**Next:** Tomorrow we'll build linear regression models for MMM!

---

## 📅 Day 66-70: Advanced MMM Topics (Condensed)

### Day 66: Linear Regression for MMM
- Ridge regression for multicollinearity
- Model validation (train/test split)
- Coefficient interpretation
- ROI calculation by channel

### Day 67: Adstock and Saturation
- Adstock transformation (geometric decay)
- Saturation curves (Hill function, log transform)
- Optimizing adstock and saturation parameters
- Diminishing returns visualization

### Day 68: Model Validation
- Train/test split for time series
- Cross-validation strategies
- Out-of-sample performance
- Model diagnostics and residual analysis

### Day 69: Budget Optimization
- Response curves by channel
- Constrained optimization
- Marginal ROI calculation
- Optimal budget allocation

### Day 70: Capstone - Build MMM Model
- End-to-end MMM project
- Data preparation through optimization
- Business recommendations
- Executive presentation

*Note: These sections would be fully expanded in a production version with detailed code examples, exercises, and mini-projects.*

---

### 🎓 Week 10 Complete!

**Congratulations!** You've mastered Marketing Mix Modeling.

**What You've Learned:**
- ✅ MMM fundamentals and use cases
- ✅ Time-series data preparation
- ✅ Regression modeling for marketing
- ✅ Adstock and saturation effects
- ✅ Budget optimization techniques

**Next Week:** Incrementality & Lift Studies - proving causality!

---